In [ ]:
# Analyse de CV PDF avec PyPDF2
# Objectif : Extraire le texte d'un CV PDF et explorer son contenu en texte brut.

In [4]:
# Installation des dépendances
!pip install PyPDF2 pdfplumber pdf2image pytesseract -q
!pip install matplotlib pillow -q
print("✅ Librairies installées")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Librairies installées



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Imports
import PyPDF2
import pdfplumber
import os
from pathlib import Path
import textwrap
import sys

print("✅ Imports terminés")

✅ Imports terminés


In [10]:
# Obtenir le répertoire racine du projet
project_root = Path().absolute().parent
data_dir = project_root / "data"

# Créer le dossier data s'il n'existe pas
if not data_dir.exists():
    print(" Création du dossier 'data'...")
    data_dir.mkdir(parents=True, exist_ok=True)
    print(" Dossier créé !")

# Lister les PDFs disponibles
pdf_files = list(data_dir.glob("*.pdf"))
print(f"\n Fichiers PDF disponibles ({len(pdf_files)}) :")

if pdf_files:
    for pdf_file in pdf_files:
        file_size = pdf_file.stat().st_size / 1024  # Taille en KB
        print(f"  - {pdf_file.name} ({file_size:.1f} KB)")
else:
    print(" Aucun CV trouvé dans 'data/'")


 Fichiers PDF disponibles (1) :
  - RESUME Robert UNG.pdf (107.3 KB)


In [11]:
#  Parser avec PyPDF2
def parse_with_pypdf2(pdf_path):
    """Extraire texte avec PyPDF2 (méthode basique)"""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f" Nombre de pages : {num_pages}")
            
            for i, page in enumerate(reader.pages):
                page_text = page.extract_text()
                text += page_text + "\n\n"
                print(f"  Page {i+1}: {len(page_text)} caractères")
        
        return text.strip()
    except Exception as e:
        print(f" Erreur avec PyPDF2 : {e}")
        return ""

# Tester sur le premier PDF trouvé
pdf_files = list(data_dir.glob("*.pdf"))
if pdf_files:
    test_pdf = pdf_files[0]
    print(f"\n Test PyPDF2 sur : {test_pdf.name}")
    text_pypdf2 = parse_with_pypdf2(test_pdf)
    
    # Afficher un extrait
    if text_pypdf2:
        print("\n Extrait (200 premiers caractères) :")
        print("-" * 50)
        print(text_pypdf2[:200])
        print("-" * 50)


 Test PyPDF2 sur : RESUME Robert UNG.pdf
 Nombre de pages : 1
  Page 1: 3176 caractères

 Extrait (200 premiers caractères) :
--------------------------------------------------
PROFILE
PROFESSIONAL EXPERIENCE EDUCATION
HARD SKILLS
LANGUAGESCorbeil-Essonnes 91100, FranceRobert.ung@telecom-sudparis.eu+33 6 04 02 12 20ROBERT UNG
Engineering student apprentice
Engineering studen
--------------------------------------------------


In [12]:
# Parser avec pdfplumber
def parse_with_pdfplumber(pdf_path):
    """Extraire texte avec pdfplumber (plus précis)"""
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            num_pages = len(pdf.pages)
            print(f" Nombre de pages : {num_pages}")
            
            for i, page in enumerate(pdf.pages):
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n\n"
                    print(f"  Page {i+1}: {len(page_text)} caractères")
                else:
                    print(f"  Page {i+1}: Aucun texte détecté (peut être une image)")
        
        return text.strip()
    except Exception as e:
        print(f" Erreur avec pdfplumber : {e}")
        return ""

# Tester pdfplumber sur le même PDF
if pdf_files:
    print(f"\n Test pdfplumber sur : {test_pdf.name}")
    text_pdfplumber = parse_with_pdfplumber(test_pdf)
    
    # Afficher un extrait
    if text_pdfplumber:
        print("\n Extrait (200 premiers caractères) :")
        print("-" * 50)
        print(text_pdfplumber[:200])
        print("-" * 50)

Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats



 Test pdfplumber sur : RESUME Robert UNG.pdf
 Nombre de pages : 1


Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats


  Page 1: 3172 caractères

 Extrait (200 premiers caractères) :
--------------------------------------------------
+33 6 04 02 12 20
ROBERT UNG
Robert.ung@telecom-sudparis.eu
Engineering student apprentice
Corbeil-Essonnes 91100, France
Portfolio website : https://robert-ung.netlify.app
PROFILE
Engineering student
--------------------------------------------------


In [13]:
# Comparaison des résultats
def compare_parsers(text1, text2, name1="PyPDF2", name2="pdfplumber"):
    """Comparer les résultats des deux parseurs"""
    print(f"\n COMPARAISON DES PARSEURS")
    print("=" * 50)
    
    # Longueur du texte
    len1 = len(text1) if text1 else 0
    len2 = len(text2) if text2 else 0
    
    print(f"{name1}: {len1} caractères")
    print(f"{name2}: {len2} caractères")
    
    # Calculer la différence
    if len1 > 0 and len2 > 0:
        diff = abs(len1 - len2)
        diff_percent = (diff / max(len1, len2)) * 100
        print(f"\nDifférence: {diff} caractères ({diff_percent:.1f}%)")
        
        # Quel est le plus long ?
        if len1 > len2:
            print(f" {name1} a extrait plus de texte")
        elif len2 > len1:
            print(f" {name2} a extrait plus de texte")
        else:
            print(f" Les deux ont extrait la même quantité")
    
    # Vérifier si les textes sont identiques
    if text1 and text2:
        if text1 == text2:
            print("\n Les textes sont IDENTIQUES")
        else:
            print("\n  Les textes sont DIFFÉRENTS")
            
            # Comparer les premiers mots
            words1 = text1.split()[:20]
            words2 = text2.split()[:20]
            print(f"\nPremiers mots {name1}: {' '.join(words1)}")
            print(f"Premiers mots {name2}: {' '.join(words2)}")

# Exécuter la comparaison
if 'text_pypdf2' in locals() and 'text_pdfplumber' in locals():
    compare_parsers(text_pypdf2, text_pdfplumber)


 COMPARAISON DES PARSEURS
PyPDF2: 3176 caractères
pdfplumber: 3172 caractères

Différence: 4 caractères (0.1%)
 PyPDF2 a extrait plus de texte

  Les textes sont DIFFÉRENTS

Premiers mots PyPDF2: PROFILE PROFESSIONAL EXPERIENCE EDUCATION HARD SKILLS LANGUAGESCorbeil-Essonnes 91100, FranceRobert.ung@telecom-sudparis.eu+33 6 04 02 12 20ROBERT UNG Engineering student apprentice Engineering student
Premiers mots pdfplumber: +33 6 04 02 12 20 ROBERT UNG Robert.ung@telecom-sudparis.eu Engineering student apprentice Corbeil-Essonnes 91100, France Portfolio website : https://robert-ung.netlify.app PROFILE


In [14]:
# Analyse du texte extrait
def analyze_text(text, parser_name):
    """Analyser le texte extrait"""
    if not text:
        print(f"\n {parser_name}: Aucun texte à analyser")
        return
    
    print(f"\n ANALYSE DU TEXTE ({parser_name})")
    print("=" * 50)
    
    # Statistiques basiques
    lines = text.split('\n')
    words = text.split()
    sentences = text.replace('!', '.').replace('?', '.').split('.')
    
    print(f"Lignes: {len(lines)}")
    print(f"Mots: {len(words)}")
    print(f"Phrases: {len(sentences)}")
    print(f"Moyenne mots/par phrase: {len(words)/max(len(sentences), 1):.1f}")
    
    # Recherche de mots-clés techniques
    technical_keywords = [
        'python', 'java', 'javascript', 'sql', 'git', 'docker',
        'machine', 'learning', 'deep', 'ai', 'data', 'analysis',
        'fastapi', 'react', 'node', 'linux', 'aws', 'cloud'
    ]
    
    print(f"\n Mots-clés techniques trouvés:")
    found_keywords = []
    text_lower = text.lower()
    
    for keyword in technical_keywords:
        if keyword in text_lower:
            found_keywords.append(keyword)
    
    if found_keywords:
        for kw in found_keywords:
            count = text_lower.count(kw)
            print(f"  - {kw}: {count} occurrence(s)")
    else:
        print("  Aucun mot-clé technique détecté")
    
    # Afficher un extrait formaté
    print(f"\n EXTRACTION RÉUSSIE !")
    print(f"Texte extrait ({len(text)} caractères)")

# Analyser les deux extractions
if 'text_pypdf2' in locals():
    analyze_text(text_pypdf2, "PyPDF2")

if 'text_pdfplumber' in locals():
    analyze_text(text_pdfplumber, "pdfplumber")


 ANALYSE DU TEXTE (PyPDF2)
Lignes: 69
Mots: 413
Phrases: 10
Moyenne mots/par phrase: 41.3

 Mots-clés techniques trouvés:
  - python: 4 occurrence(s)
  - java: 1 occurrence(s)
  - sql: 1 occurrence(s)
  - git: 1 occurrence(s)
  - docker: 1 occurrence(s)
  - machine: 4 occurrence(s)
  - learning: 7 occurrence(s)
  - deep: 2 occurrence(s)
  - ai: 6 occurrence(s)
  - data: 12 occurrence(s)
  - analysis: 1 occurrence(s)
  - linux: 1 occurrence(s)

 EXTRACTION RÉUSSIE !
Texte extrait (3176 caractères)

 ANALYSE DU TEXTE (pdfplumber)
Lignes: 58
Mots: 418
Phrases: 10
Moyenne mots/par phrase: 41.8

 Mots-clés techniques trouvés:
  - python: 4 occurrence(s)
  - java: 1 occurrence(s)
  - sql: 1 occurrence(s)
  - git: 1 occurrence(s)
  - docker: 1 occurrence(s)
  - machine: 4 occurrence(s)
  - learning: 7 occurrence(s)
  - deep: 2 occurrence(s)
  - ai: 6 occurrence(s)
  - data: 12 occurrence(s)
  - analysis: 1 occurrence(s)
  - linux: 1 occurrence(s)

 EXTRACTION RÉUSSIE !
Texte extrait (3172 ca

In [15]:
# Créer une fonction utilitaire réutilisable
class CVParser:
    """Parser de CV PDF avec choix de méthode"""
    
    def __init__(self, method='pdfplumber'):
        """
        Initialiser le parser
        Args:
            method: 'pdfplumber' (par défaut) ou 'pypdf2'
        """
        self.method = method
        self.text = ""
        
    def parse(self, pdf_path):
        """Parser un fichier PDF"""
        pdf_path = Path(pdf_path)
        
        if not pdf_path.exists():
            print(f" Fichier non trouvé : {pdf_path}")
            return ""
        
        print(f" Parsing de : {pdf_path.name}")
        print(f" Méthode : {self.method}")
        
        if self.method == 'pypdf2':
            self.text = parse_with_pypdf2(pdf_path)
        elif self.method == 'pdfplumber':
            self.text = parse_with_pdfplumber(pdf_path)
        else:
            print(f" Méthode non supportée : {self.method}")
            self.text = ""
        
        return self.text
    
    def get_stats(self):
        """Obtenir des statistiques sur le texte extrait"""
        if not self.text:
            return {"error": "Aucun texte extrait"}
        
        return {
            "characters": len(self.text),
            "words": len(self.text.split()),
            "lines": len(self.text.split('\n')),
            "method": self.method
        }
    
    def save_text(self, output_path=None):
        """Sauvegarder le texte extrait"""
        if not self.text:
            print(" Aucun texte à sauvegarder")
            return False
        
        if output_path is None:
            output_path = f"cv_text_{self.method}.txt"
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(self.text)
        
        print(f" Texte sauvegardé dans : {output_path}")
        return True

# Tester la classe
if pdf_files:
    print("\n TEST DE LA CLASSE CVParser")
    print("=" * 50)
    
    # Tester avec pdfplumber
    parser1 = CVParser(method='pdfplumber')
    text1 = parser1.parse(test_pdf)
    print(f"Statistiques: {parser1.get_stats()}")
    parser1.save_text()
    
    # Tester avec pypdf2
    parser2 = CVParser(method='pypdf2')
    text2 = parser2.parse(test_pdf)
    print(f"Statistiques: {parser2.get_stats()}")
    parser2.save_text()

Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats



 TEST DE LA CLASSE CVParser
 Parsing de : RESUME Robert UNG.pdf
 Méthode : pdfplumber
 Nombre de pages : 1


Could not get FontBBox from font descriptor because None cannot be parsed as 4 floats


  Page 1: 3172 caractères
Statistiques: {'characters': 3172, 'words': 418, 'lines': 58, 'method': 'pdfplumber'}
 Texte sauvegardé dans : cv_text_pdfplumber.txt
 Parsing de : RESUME Robert UNG.pdf
 Méthode : pypdf2
 Nombre de pages : 1
  Page 1: 3176 caractères
Statistiques: {'characters': 3176, 'words': 413, 'lines': 69, 'method': 'pypdf2'}
 Texte sauvegardé dans : cv_text_pypdf2.txt
